In [15]:
import numpy as np
import scipy.special as bessel
from scipy.optimize import curve_fit
from scipy.optimize import least_squares

import math as maths
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import pickle
import matplotlib.cm as cm
from random import randrange



In [ ]:
def f(x, A, B): 
    return A*x + B

def inverted_v(input_data, L):
    output = np.log(1-np.exp(-1/L)) - np.log(1+np.exp(-1/L)) -np.abs(np.array(input_data))/L
    return output





The quantum state is represented by a vector in momentum space. The state at t=0 is the initial condition, and we can calculate how the state evolves in time for discrete values of the period (T) of the system, i.e. we can know the state of the system at t=2T, but not t=2.5T.

The system has 2 free parameters, k and tau. 

For more details see https://doi.org/10.1063/5.0084028

vector(t=t_0+T) = vector(t=t_0)*U_matrix

In [ ]:

#This function calculates the U_matrix for a certain value of k, tau. 
#The size of the matrix is also important. Make it as large as possible
def gbg_method(matrix_size, k, tau):
    #matrix_size must be odd
    N = int(np.floor(matrix_size/2))
    l = list(range(-N, N+1))
    G = np.zeros((len(l), len(l)), dtype = "complex")
    B_lower_diag = np.zeros((len(l), len(l)), dtype = "complex")
    for i in range(0, len(l)):
        G[i,i] = np.exp(-1*complex(0,1)*(tau/4)*l[i]**2)
        for j in range(0, i+1):
            B_lower_diag[i,j]=(complex(0,1)**(l[i]-l[j]))*bessel.jv(l[i]-l[j], k)
    B = B_lower_diag + B_lower_diag.T - np.diag(B_lower_diag.diagonal())
    GBG = np.matmul(G, np.matmul(B, G))
    return GBG

#This function generates the initial conditions
def initial_condition_generator(size, list_input): #list_input = [mag, element1, element2, ...]
    """
    Input the length of the desired vector (size). All elements of the vector will default to 0, unless specified
    in the list_input
    
    output:
    single normalised vector 
    
    """
    initial_condition = np.zeros((size, 1))
    mag = list_input[0]
    elements = list_input[1:]
    for element in elements:
        initial_condition[element] = mag
    norm = np.linalg.norm(initial_condition)
    initial_condition = initial_condition/norm
    return initial_condition


def matrix_sizer_single(k, tau, size, small_number_input):
    x_axis_data = []
    eig_values_size = []
    U = gbg_method(size, k, tau)
    x_axis_data.append(size)
    eigenvalues = np.flip(np.sort(np.abs(np.linalg.eig(U)[0])))
    counter = 0
    for element in eigenvalues:
        if np.abs(1 - element) > small_number_input:
            counter = counter + 1
    ratio_item = round((1 - counter/len(eigenvalues))*100, 2)
    return ratio_item


def quantum_kicker(n, k, tau, steps, initial_condition):
    """
    n = side-size of matrix U. Must be an odd integer
    steps = number of kicks that will be simulated
    """
    #calculate matrix U
    U = gbg_method_better(n, k, tau)
    traj = initial_condition
    vector = initial_condition
    #Simply the new vector is equal to the old one multiplied by the U matrix (the U matrix is constant)
    for step in range(steps-1):
        #new_vector = old_vector*U_matrix
        vector = U.dot(vector)
        traj = np.concatenate((traj, vector), 1)
    #traj is simply the collection of vectors. Each vector describes the state.
    return traj

#U_steps = np.linalg.matrix_power(U, steps)

def analyser(input_array):
    length = input_array.shape[1]
    output = []
    for i in range(length):
        output.append(input_array[:, i])
    return np.array(output)

def zero_finder(input_list):
    output_list = []
    for data_list in input_list:
        indices = []
        for i in range(len(data_list)-1):
            if (data_list[i] == 0 and data_list[i+1] != 0) or (data_list[i] != 0 and data_list[i+1] == 0):
                indices.append(i+1)
        if indices == []:
            indices = [0, len(data_list)]
        output_list.append(indices)
    return output_list

#This function outputs the energy of each vector when the trajectory is inputted
def energy_analyser(input_coefficients_list):
    l = len(input_coefficients_list)
    basis_list = np.array(list(range(-l//2 +1, (l//2)+1)))
    basis_list_sqr = basis_list**2
    coefficients_sqr = np.abs(input_coefficients_list)**2
    energy = 0.5*sum(basis_list_sqr*coefficients_sqr)
    #The energy is given by (1/2)*SUM_OVER_i{(coefficients_i^2)*(basis_element_i^2)}
    return energy




In [ ]:
Analysis


In [ ]:
ll_data = []
fraction_to_cut = 0.4

for i in range(len(data_in)):
    K_data = []
    print("K = "+str(K_numbers[i+K_offset]))
    for j in range(len(k_list)):
        print(j)
        k_data = []
        full_x_data = data_in[i][j][0]
        x_half_len = len(full_x_data)//2
        for frame in range(1, len(data_in[0][0][1])):
            #This finds the indices which correspond to 0. Sometimes there are more than 2 e.g. [0, 0, 0.1, 0, 0.2, 1, 0.2, 0, 0, 0]
            zero_indices = zero_finder([list(np.abs(data_in[i][j][1][frame])**2)])[0]
            #This selects the indices which are the two inner ones
            selected_indices = selector(zero_indices, x_half_len*2)
            #This selects the zero_index closer to the centre
            smaller_z_i = min([x_half_len-selected_indices[0], selected_indices[1]-x_half_len])
            #Full dataset with 0s removed
            unsliced_x = data_in[i][j][0][x_half_len-ploffset: x_half_len+ploffset]
            unsliced_y = np.log([list(np.abs(data_in[i][j][1][frame])**2)][0][x_half_len-ploffset: x_half_len+ploffset])
            #This selects the indices which will be sliced
            slicer_indices = conceptual_divider(len(unsliced_x), fraction_to_cut)
            #Middle fraction sliced out
            x_data = slicer(unsliced_x, slicer_indices)
            y_data = slicer(unsliced_y, slicer_indices)
            ####
            if frame != 1 and v_popt < 9:
                suggestion = v_popt*1.01
            else:
                suggestion = 0.2
            v_popt, v_pcov = curve_fit(inverted_v_2, x_data, y_data, p0 = suggestion, bounds = (0.05, 10))
            k_data.append(v_popt)
        K_data.append(k_data)
    ll_data.append(K_data)
    


In [ ]:
energy_data = []
for K_data in data_k_tau_grid1:
    big_temp = []
    for k_data in K_data:
        temp = [energy_analyser(vector) for vector in k_data[1]]
        big_temp.append(temp)
    energy_data.append(big_temp)

In [4]:
def choose(n, k):
    return maths.factorial(n)/(maths.factorial(k)*maths.factorial(n-k))

In [9]:
count = 0
for i in range(8, 11):
    count = count + choose(10, i)*(0.8**i)*(0.2**(10-i))

In [10]:
print(count)

0.6777995264000004


In [125]:
count = 0
for i in range(0, 8):
    count = count + choose(10, i)*(0.8**i)*(0.2**(10-i))

In [126]:
print(count)

0.32220047360000015


In [48]:
#men = 0
#women = 0
all_people = []
for i in range(50):
    all_people.append(0)
    all_people.append(1)

In [49]:
print(all_people)

[0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1]


In [105]:


def exercise():
    indices_list = []
    results = []
    for i in range(4):
        n = 1
        temp = []
        while n<26:
            index = randrange(0, 100)
            if index not in indices_list:
                indices_list.append(index)
                temp.append(all_people[index])
                n = n+1
        results.append(temp)
    return results


In [107]:
l = exercise()
print(l)

[[1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0], [1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1], [1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0], [0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0]]


In [88]:
print(results)

[[0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1], [1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0], [1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0], [1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0]]


In [127]:

fin = []
for repetition in range(500000):
    final = []
    sub_sum_list = []
    results = exercise()
    for sub_list in results:
        sub_sum = sum(sub_list)
        sub_sum_list.append(sub_sum)
    #print(total)
    final.append(max(sub_sum_list))
    fin.append(final[0])

    

In [128]:
print(np.mean(fin))
print(np.median(fin))

15.070142
15.0


In [ ]:
15.07488 for 100,000
15.071904 for 500,000

In [123]:
print(max(fin))

23
